## Imports

In [1]:
#export
import numpy as np
import os
import torch
from torch.utils.data import DataLoader
from torch import tensor
from torchvision import transforms
from PIL import Image
import warnings
from tqdm.auto import tqdm

## Get

In [2]:
#export
def get_datasets(god):
    if not os.path.isdir(god.config['dataset']['path']):
        raise Exception(f"Dataset at path '{god.config['dataset']['path']}' was not found, make sure that the provided path in config.dataset.path is correct.")
    
    ds_type = god.config['dataset']['type']
    if ds_type == 'ImageDir':
        return get_ImageDirDataset(god)
    if ds_type == 'ImageDirInMemory':
        return get_ImageDirInMemoryDataset(god)
    elif ds_type == 'IdxImageDir':
        return get_IdxImageDirDataset(god)
    if ds_type == 'ImageFileName':
        return get_ImageFileNameDataset(god)
    raise Exception(f"dataset type '{ds_type}' not supported")

In [3]:
#export
def get_dataloaders(god):
    return {
        'train': DataLoader(god.datasets['train'], batch_size=god.config['hp']['batch_size'], num_workers=get_num_workers(god), sampler=god.samplers['train']),
        'val': DataLoader(god.datasets['val'], batch_size=god.config['hp']['batch_size'], num_workers=get_num_workers(god), sampler=god.samplers['val'])
    }

In [4]:
#export 
def get_num_workers(god):
    if 'num_workers' in god.config['constants']:
        return god.config['constants']['num_workers']
    if os.cpu_count() > 0:
        return os.cpu_counts()
    else:
        warnings.warn('Could not determine number of cores via os.cpu_count(), using 4 as default')
        return 4

In [5]:
#export
def get_data_transforms(god):
    data_transforms = {}
    for phase in ('train', 'val'):
        phase_transforms = [get_data_transform(god, t) for t in god.config['data_transforms'][phase]]
        data_transforms[phase] = transforms.Compose([t for t in phase_transforms if t is not None])
    return data_transforms

In [6]:
#export
def get_mean_std_per_channel(god):
    n, total_sum, total_square_sum = 0, tensor([0.,0.,0.]), tensor([0.,0.,0.])
    for xb, yb in god.dataloaders['train']:
        n += xb.numel() // xb.shape[1]  # Number of samples, excluding channel dimension
        total_sum += xb.sum(dim=(0,2,3))
        total_square_sum += (xb**2).sum(dim=(0,2,3))

    mean = total_sum / n
    std_dev = torch.sqrt((total_square_sum / n) - mean**2)
    return mean.tolist(), std_dev.tolist()

In [7]:
#export
def get_class_counts(god):
    class_counts = {}
    for phase in ('train', 'val'):
        dl = god.dataloaders[phase]
        indices = list(dl.sampler)
        targets = torch.tensor([dl.dataset.get_y(i) for i in indices])
        class_counts[phase] = targets.bincount().int().tolist()
    return class_counts

## Get Util

In [8]:
#export
# TODO combine the similar approaches and just change the dataset class


def get_ImageDirDataset(god):
    return {
        'train': ImageDirDataset.from_directory(
            os.path.join(god.config['dataset']['path'], god.config['dataset']['train_path']),
            god.config['label2id'],
            transform=god.data_transforms['train']
        ),
        'val':  ImageDirDataset.from_directory(
            os.path.join(god.config['dataset']['path'], god.config['dataset']['val_path']),
            god.config['label2id'],
            transform=god.data_transforms['val']
        )
    }

def get_ImageDirInMemoryDataset(god):
    return {
        'train': ImageDirInMemoryDataset(
            os.path.join(god.config['dataset']['path'], god.config['dataset']['train_path']),
            god.config['label2id'],
            transform=god.data_transforms['train']
        ),
        'val':  ImageDirInMemoryDataset(
            os.path.join(god.config['dataset']['path'], god.config['dataset']['val_path']),
            god.config['label2id'],
            transform=god.data_transforms['val']
        )
    }

def get_IdxImageDirDataset(god):
    img_dir_ds =  ImageDirDataset.from_directory(god.config['dataset']['path'], god.config['label2id'])

    val_pct = god.config['dataset']['val_pct']
    length = len(img_dir_ds)
    val_size = int(val_pct * length)

    perm = torch.randperm(length)
    train_idxs = perm[val_size:]
    val_idxs = perm[:val_size]
    
    return {
        'train': IdxImageDirDataset(img_dir_ds, train_idxs, god.config['data_transforms']['train']),
        'val':  IdxImageDirDataset(img_dir_ds, val_idxs, god.config['data_transforms']['val'])
    }

def get_ImageFileNameDataset(god):
    return {
        'train': ImageFileNameDataset.from_directory(
            os.path.join(god.config['dataset']['path'], god.config['dataset']['train_path']),
            god.config['dataset']['filename2label'],
            god.config['label2id'],
            god.data_transforms['train']
        ),
        'val': ImageFileNameDataset.from_directory(
            os.path.join(god.config['dataset']['path'], god.config['dataset']['val_path']),
            god.config['dataset']['filename2label'],
            god.config['label2id'],
            god.data_transforms['val']
        )
}

In [9]:
#export
def get_data_transform(god, transform_dict):
    if transform_dict['name'] == 'SquarePad':
        extra_pad = transform_dict['extra_pad'] if 'extra_pad' in transform_dict else 0
        return SquarePad(extra_pad=extra_pad)
    
    elif transform_dict['name'] == 'Resize':
        return transforms.Resize(transform_dict['size'])
        
    elif transform_dict['name'] == 'ToTensor':
        return transforms.ToTensor()
        
    elif transform_dict['name'] == 'Grayscale':
        num_output_channels = transform_dict['num_output_channels'] if 'num_output_channels' in transform_dict else 1
        return transforms.Grayscale(num_output_channels=num_output_channels)
    
    elif transform_dict['name'] == 'Cutout':
        return Cutout(transform_dict['n_holes'], transform_dict['length'])
    
    elif transform_dict['name'] == 'CenterCrop':
        return transforms.CenterCrop(transform_dict['size'])
    
    # TODO add support for remaining settings
    elif transform_dict['name'] == 'RandomCrop':
        return transforms.RandomCrop(transform_dict['size'])
    
    elif transform_dict['name'] == 'RandomHorizontalFlip':
        return transforms.RandomHorizontalFlip(transform_dict.get('p', 0.5))
    
    elif transform_dict['name'] == 'RandomVerticalFlip':
        return transforms.RandomVerticalFlip(transform_dict.get('p', 0.5))
    
    # TODO add support for remaining settings
    elif transform_dict['name'] == 'RandomRotation':
        return transforms.RandomRotation(transform_dict['degrees'])
    
    # TODO add support for remaining settings
    elif transform_dict['name'] == 'RandomResizedCrop':
        return transforms.RandomResizedCrop(transform_dict['size'])
        
    elif transform_dict['name'] == 'Normalize':
        if not 'x_mean' in god.config['constants'] or not 'x_std' in god.config['constants']:
            warnings.warn('Normalize Transform will not be used since it needs config.constants.x_mean and config.constants.x_std to be set. They can be obtained by calling god.print_x_mean_and_std().')
            return None
            
        return transforms.Normalize(god.config['constants']['x_mean'], god.config['constants']['x_std'])
    
    raise Exception(f"Transform with name '{transform_dict['name']}' is not supported.")        

## Datasets

In [10]:
#export
class ImageDirDataset():
    @staticmethod
    def from_directory(image_directory, label2id, transform=transforms.Compose([]), in_memory=False, image_extensions=['.png', '.jpg', '.jpeg']):
            image_label_tuples = []
            for label in os.listdir(image_directory):
                if label.startswith('IGNORE_LABEL') or label.startswith('.') : continue
                for image in os.listdir(os.path.join(image_directory, label)):
                    if any(image.endswith(ext) for ext in image_extensions):
                        image_label_tuples.append((image, label))
                        
            return ImageDirDataset(image_directory, image_label_tuples, label2id, transform)
        
    
    def __init__(self, img_dir, image_label_tuples, label2id, transform=transforms.Compose([])):
        self.img_dir, self.image_label_tuples, self.label2id, self.transform = img_dir, image_label_tuples, label2id, transform
        self[0]
        
    def __getitem__(self, idx):
        #import pdb; pdb.set_trace()
        if type(idx) != int: raise Exception(f'Only int index support for now, got idx of type "{type(idx).__name__}"')
        img_name, img_label = self.image_label_tuples[idx]
        img_path = os.path.join(self.img_dir, img_label, img_name)
        return self.transform(Image.open(img_path).convert('RGBA').convert('RGB')), self.label2id[img_label]
    
    
    def get_y(self, idx):
        if type(idx) != int: raise Exception(f'Only int index support for now, got idx of type "{type(idx).__name__}"')
        return self.label2id[self.image_label_tuples[idx][1]]

    def __len__(self):
        return len(self.image_label_tuples)  

In [11]:
#export
class ImageDirInMemoryDataset():
    def __init__(self, img_dir, label2id, transform=transforms.Compose([]), image_extensions=['.png', '.jpg', '.jpeg']):
        self.img_dir = img_dir
        self.label2id = label2id
        self.transform = transform
        self.image_extensions = image_extensions

        self.images = []
        self.labels = []

        self._load_images()

    def _load_images(self):
        for label in tqdm(os.listdir(self.img_dir)):
            if label.startswith('IGNORE_LABEL') or label.startswith('.') : continue
            for image in tqdm(os.listdir(os.path.join(self.img_dir, label))):
                if any(image.endswith(ext) for ext in self.image_extensions):
                    image_path = os.path.join(self.img_dir, label, image)
                    image_tensor = self.transform(Image.open(image_path).convert('RGBA').convert('RGB'))
                    self.images.append(image_tensor)
                    self.labels.append(self.label2id[label])

        # Convert lists of images and labels to tensors
        #self.images = torch.stack(self.images)
        #self.labels = torch.tensor(self.labels)
        
    def __getitem__(self, idx):
        if type(idx) != int: raise Exception(f'Only int index support for now, got idx of type "{type(idx).__name__}"')
        return self.images[idx], self.labels[idx]

    def __len__(self):
        return len(self.images)


In [12]:
#export
class ImageDirDataset():
    @staticmethod
    def from_directory(image_directory, label2id, transform=transforms.Compose([]), image_extensions=['.png', '.jpg', '.jpeg']):
            image_label_tuples = []
            for label in os.listdir(image_directory):
                if label.startswith('IGNORE_LABEL') or label.startswith('.') : continue
                for image in os.listdir(os.path.join(image_directory, label)):
                    if any(image.endswith(ext) for ext in image_extensions):
                        image_label_tuples.append((image, label))
                        
            return ImageDirDataset(image_directory, image_label_tuples, label2id, transform)
        
    
    def __init__(self, img_dir, image_label_tuples, label2id, transform=transforms.Compose([])):
        self.img_dir, self.image_label_tuples, self.label2id, self.transform = img_dir, image_label_tuples, label2id, transform
        self[0]
        
    def __getitem__(self, idx):
        #import pdb; pdb.set_trace()
        if type(idx) != int: raise Exception(f'Only int index support for now, got idx of type "{type(idx).__name__}"')
        img_name, img_label = self.image_label_tuples[idx]
        img_path = os.path.join(self.img_dir, img_label, img_name)
        return self.transform(Image.open(img_path).convert('RGBA').convert('RGB')), self.label2id[img_label]
    
    
    def get_y(self, idx):
        if type(idx) != int: raise Exception(f'Only int index support for now, got idx of type "{type(idx).__name__}"')
        return self.label2id[self.image_label_tuples[idx][1]]

    def __len__(self):
        return len(self.image_label_tuples)  

In [13]:
#export
# wrapper for ImageDirDataset to support custom indices    
class IdxImageDirDataset():
    def __init__(self, image_dir_ds, idxs, transform=transforms.Compose([])):
        self.ds, self.idxs, self.transform = image_dir_ds, idxs, transform
    
    def __getitem__(self, idx): 
        x,y = self.ds[self.idxs[idx].item()]
        return self.transform(x), y
    
    def get_y(self, idx):
        if type(idx) != int: raise Exception(f'Only int index support for now, got idx of type "{type(idx).__name__}"')
        return self.ds.get_y(self.idxs[idx].items())
    
    def __len__(self):
        return len(self.idxs)    

In [14]:
#exprt
class ImageFileNameDataset():
    @staticmethod
    def from_directory(image_directory, filename2label, label2id, transform=transforms.Compose([]), image_extensions=['.png', '.jpg', '.jpeg']):
        image_label_tuples = []
        for image_name in os.listdir(image_directory):
            if not any(image_name.endswith(ext) for ext in image_extensions):
                continue
            image_file = os.path.join(image_directory, image_name)
            image_label = filename2label(image_name)
            image_label_tuples.append((image_name, image_label))
            
        return ImageFileNameDataset(image_directory, image_label_tuples, label2id, transform)
        
    def __init__(self, image_directory, image_label_tuples, label2id, transform=transforms.Compose([])):
        self.image_directory, self.image_label_tuples, self.label2id, self.transform = image_directory, image_label_tuples, label2id, transform

    def __getitem__(self, idx):
        if type(idx) != int: raise Exception(f'Only int index support for now, got idx of type "{type(idx).__name__}"')
        img_name, img_label = self.image_label_tuples[idx]
        img_path = os.path.join(self.image_directory, img_name)
        return self.transform(Image.open(img_path).convert('RGBA').convert('RGB')), self.label2id[img_label]
    
    def get_y(self, idx):
        if type(idx) != int: raise Exception(f'Only int index support for now, got idx of type "{type(idx).__name__}"')
        return self.label2id[self.image_label_tuples[idx][1]]
    
    def __len__(self):
        return len(self.image_label_tuples)         

## Data Transforms

In [15]:
#export
class SquarePad:
    def __init__(self, extra_pad=0): self.extra_pad = extra_pad
    
    def __call__(self, image):
        s = image.size
        max_wh = np.max([s[-1], s[-2]])
        hp = int((max_wh - s[-2]) / 2) + self.extra_pad
        vp = int((max_wh - s[-1]) / 2) + self.extra_pad
        padding = (hp, vp)
        return transforms.Pad(padding, 0, 'constant')(image)

In [16]:
#export
class Cutout(object):
    """Randomly mask out one or more patches from an image.
    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img

## Data Util

In [ ]:
# OLD
def calc_mean_std_per_channel(god):
    n, total_mean, total_std = 0, tensor([0.,0.,0.]), tensor([0.,0.,0.])
    for dl in god.dataloaders.values():
        for xb,yb in dl:
            n += len(xb)
            total_mean.add_(xb.mean(dim=(0,2,3)) * len(xb))
            total_std.add_(xb.std(dim=(0,2,3)) * len(xb))
        
    return (total_mean / n).tolist(), (total_std / n).tolist()

## Test+Export

In [ ]:
!python _notebook2script.py 02_Data.ipynb